In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import h_key
from api_keys import f_key


# Output File (CSV)
output_data_file = "output_data/hotel.csv"

# Range of latitudes and longitudes
#lat_range = (-90, 90)
#lng_range = (-180, 180)

### API Calls

In [2]:
# Create project dataframe (cities)
city_df = pd.read_csv("./Resources/Airport_Output.csv")
city_df.dropna(inplace = True) 
city_df.head()


,Unnamed: 0,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating,Airport Code
0,0,New York City,New York,40.712775,-74.005973,Newark Liberty International Airport,"3 Brewster Rd, Newark",3.2,EWR
1,1,Los Angeles,California,34.052234,-118.243685,Los Angeles International Airport,"1 World Way, Los Angeles",3.5,LAX
2,2,Chicago,Illinois,41.878114,-87.629798,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",3.6,ORD
3,3,Houston,Texas,29.760427,-95.369803,William P. Hobby Airport,"7800 Airport Blvd, Houston",4.0,HOU
4,4,Philadelphia,Pennsylvania,39.952584,-75.165222,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.3,PHL


In [3]:
airport_code=[]
cheapestTicketPrice=[]
#outbound trip
for index, row in city_df.iterrows():
    code = row['Airport Code']
    originplace='BWI'
    destinationplace=code
    departure_date="2021-03-15"
    base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/"
    tags=originplace + "-sky/" + destinationplace + "-sky/" + departure_date
    url=base_url + tags
    print(url)
    querystring = {"inboundpartialdate":"2021-03-19"}

    headers = {
        'x-rapidapi-key': "e5ee88badfmsh65ec7588e95e6fcp181b1cjsn38212393a000",
        'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com"
        }
    try:
        response = requests.request("GET", url, headers=headers, params=querystring)
        response=response.json()
        airport_code.append(code)
        cheapestTicketPrice.append(response['Quotes'][0]['MinPrice'])
    except:
        cheapestTicketPrice.append('nan')


https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/EWR-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/LAX-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/ORD-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/HOU-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/PHL-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/PHX-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/SAT-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/br

https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/GSO-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/DFW-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/EWR-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/TOL-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/LINK-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/MCO-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/BWI-sky/SAN-sky/2021-03-15
https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/b

In [4]:
city_df['Cheapest_Outbound_Airfare']=cheapestTicketPrice

In [5]:
airport_code=[]
cheapestTicketPrice=[]
#return trip
for index, row in city_df.iterrows():
    code = row['Airport Code']
    destinationplace='IAD'
    originplace=code
    departure_date="2021-03-19"
    base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/"
    tags=originplace + "-sky/" + destinationplace + "-sky/" + departure_date
    url=base_url + tags
    #print(url)
    querystring = {"inboundpartialdate":"2021-03-19"}

    headers = {
        'x-rapidapi-key': "e5ee88badfmsh65ec7588e95e6fcp181b1cjsn38212393a000",
        'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com"
        }
    try:
        response = requests.request("GET", url, headers=headers, params=querystring)
        response=response.json()
        airport_code.append(code)
        cheapestTicketPrice.append(response['Quotes'][0]['MinPrice'])
    except:
        cheapestTicketPrice.append('nan')

In [6]:
city_df['Cheapest_Return_Airfare']=cheapestTicketPrice
city_df

#cleaned_city_df.to_csv("./Resources/Airport_Prices.csv", index=False, header=True)

,Unnamed: 0,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating,Airport Code,Cheapest_Outbound_Airfare,Cheapest_Return_Airfare
0,0,New York City,New York,40.712775,-74.005973,Newark Liberty International Airport,"3 Brewster Rd, Newark",3.2,EWR,nan,nan
1,1,Los Angeles,California,34.052234,-118.243685,Los Angeles International Airport,"1 World Way, Los Angeles",3.5,LAX,nan,nan
2,2,Chicago,Illinois,41.878114,-87.629798,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",3.6,ORD,nan,nan
3,3,Houston,Texas,29.760427,-95.369803,William P. Hobby Airport,"7800 Airport Blvd, Houston",4.0,HOU,nan,nan
4,4,Philadelphia,Pennsylvania,39.952584,-75.165222,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.3,PHL,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...
94,94,North Las Vegas,Nevada,36.198859,-115.117501,McCarran International Airport,"5757 Wayne Newton Blvd, Las Vegas",3.9,LAS,nan,nan
95,95,Fremont,California,37.548270,-121.988572,San Francisco International Airport,"802 E Atherton Dr, Manteca",4.1,SFO,nan,nan
96,96,Baton Rouge,Louisiana,30.451468,-91.187147,Baton Rouge Metropolitan Airport,"9430 Jackie Cochran Dr, Baton Rouge",3.8,BTR,nan,nan
97,97,Richmond,Virginia,37.540725,-77.436048,Richmond International Airport,"1 Richard E Byrd Terminal Dr, Richmond",4.1,RIC,nan,nan


In [7]:
#Kayla API

import requests


url = "https://hotels-com-free.p.rapidapi.com/srle/listing/v1/brands/hotels.com"

querystring = {"lat":"37.788719679657554","lon":"-122.40057774847898","checkIn":"2021-03-15","checkOut":"2021-03-19","rooms":"2","locale":"en_US","currency":"USD","pageNumber":"1"}

headers = {
    'x-rapidapi-key': h_key,
    'x-rapidapi-host': "hotels-com-free.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

response.json()

response = response.json()


{"message":"You have exceeded the DAILY quota for Requests on your current plan, BASIC. Upgrade your plan at https:\/\/rapidapi.com\/yahoo.finance.low.latency\/api\/hotels-com-free"}


In [8]:
response["data"]

KeyError: 'data'

In [ ]:
counter = 0
cities = []
results=[]

    

#For loop
for city,item in city_df.iterrows():
    counter=counter+1  
    #print(item)
    try:
        url = "https://hotels-com-free.p.rapidapi.com/srle/listing/v1/brands/hotels.com"
        #print(item["Lat"],item["Lng"]) 
        querystring = {"lat":item["Lat"],"lon":item["Lng"],"checkIn":"2021-03-15","checkOut":"2021-03-19","rooms":"2","locale":"en_US","currency":"USD","pageNumber":"1"}

        headers = {
        'x-rapidapi-key': h_key,
        'x-rapidapi-host': "hotels-com-free.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)

        response = response.json()
    

        nh = len(response['data']['body']['searchResults']['results'])

        for i in range(nh):

            Hotel = response['data']['body']['searchResults']['results'][i]['name']
            Rating = response['data']['body']['searchResults']['results'][i]['starRating']
            City = response['data']['body']['searchResults']['results'][i]['address']['locality']
            State = response['data']['body']['searchResults']['results'][i]['address']['region']
            Country =response['data']['body']['searchResults']['results'][i]['address']['countryCode']
            Stay_Rate = response['data']['body']['searchResults']['results'][i]['ratePlan']['price']['exactCurrent']
            
            results.append([Hotel, Rating,City, State, Country,Stay_Rate])
            
    except:
        
        print("Hotel Data Not Found")
   
               
print(results)


In [ ]:
hotel_df = pd.DataFrame(results,columns=["Hotel","Rating","City", "State", "Country","Price"])

hotel_df

hotel_df.to_csv('HOTEL_API.csv', index=False)